In [6]:
import _init_paths
from lib.config import cfg
from lib.config import update_config
update_config(cfg)
from lib.common.humanKeypoints import humanKeypoints

infantKeypoint = humanKeypoints('infant.mov')
variability = infantKeypoint.getKeypointVariability()
